# Finetuning efficientNet

Lets try the model that is trending for image classification on HuggingFace: efficientnet_b2.ra_in1k

In [1]:
from datasets import load_dataset

cifar10dataset = load_dataset("cifar10", split="train[:5000]")

In [2]:
cifar10dataset = cifar10dataset.train_test_split(test_size=0.2)

In [3]:
labels = cifar10dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [4]:
from transformers import AutoImageProcessor

#import timm
#model = timm.create_model("hf_hub:timm/efficientnet_b2.ra_in1k", pretrained=True)

checkpoint = "google/efficientnet-b0"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

2023-08-10 16:29:45.651949: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 16:29:46.407135: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [6]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["img"]]
    del examples["img"]
    return examples

In [7]:
cifar10dataset = cifar10dataset.with_transform(transforms)

In [8]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [9]:
import evaluate

accuracy = evaluate.load("accuracy")

In [10]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [11]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    ignore_mismatched_sizes=True,
    id2label=id2label,
    label2id=label2id,
)

Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b0 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1280]) in the checkpoint and torch.Size([10, 1280]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import torch 
torch.cuda.empty_cache()


In [13]:
training_args = TrainingArguments(
    output_dir="cifar10_efficientnet",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8, # memory error with 16
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=cifar10dataset["train"],
    eval_dataset=cifar10dataset["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/mahnaz/mlprojects/bloom-classifier/venv_bloom-classifier/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/375 [00:00<?, ?it/s]

{'loss': 2.3032, 'learning_rate': 1.3157894736842106e-05, 'epoch': 0.08}
{'loss': 2.3025, 'learning_rate': 2.6315789473684212e-05, 'epoch': 0.16}
{'loss': 2.3024, 'learning_rate': 3.9473684210526316e-05, 'epoch': 0.24}
{'loss': 2.3007, 'learning_rate': 4.9703264094955494e-05, 'epoch': 0.32}
{'loss': 2.2981, 'learning_rate': 4.821958456973294e-05, 'epoch': 0.4}
{'loss': 2.2938, 'learning_rate': 4.673590504451038e-05, 'epoch': 0.48}
{'loss': 2.2915, 'learning_rate': 4.525222551928784e-05, 'epoch': 0.56}
{'loss': 2.2869, 'learning_rate': 4.3768545994065286e-05, 'epoch': 0.64}
{'loss': 2.2851, 'learning_rate': 4.228486646884273e-05, 'epoch': 0.72}
{'loss': 2.2755, 'learning_rate': 4.080118694362018e-05, 'epoch': 0.8}
{'loss': 2.2701, 'learning_rate': 3.9317507418397627e-05, 'epoch': 0.88}
{'loss': 2.268, 'learning_rate': 3.783382789317508e-05, 'epoch': 0.96}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.252371072769165, 'eval_accuracy': 0.334, 'eval_runtime': 4.0865, 'eval_samples_per_second': 244.711, 'eval_steps_per_second': 30.589, 'epoch': 1.0}
{'loss': 2.252, 'learning_rate': 3.635014836795252e-05, 'epoch': 1.04}
{'loss': 2.2462, 'learning_rate': 3.4866468842729974e-05, 'epoch': 1.12}
{'loss': 2.2367, 'learning_rate': 3.338278931750742e-05, 'epoch': 1.2}
{'loss': 2.2322, 'learning_rate': 3.189910979228487e-05, 'epoch': 1.28}
{'loss': 2.2267, 'learning_rate': 3.0415430267062318e-05, 'epoch': 1.36}
{'loss': 2.221, 'learning_rate': 2.8931750741839762e-05, 'epoch': 1.44}
{'loss': 2.1939, 'learning_rate': 2.744807121661721e-05, 'epoch': 1.52}
{'loss': 2.2004, 'learning_rate': 2.5964391691394662e-05, 'epoch': 1.6}
{'loss': 2.1941, 'learning_rate': 2.4480712166172106e-05, 'epoch': 1.68}
{'loss': 2.1912, 'learning_rate': 2.2997032640949558e-05, 'epoch': 1.76}
{'loss': 2.1689, 'learning_rate': 2.1513353115727002e-05, 'epoch': 1.84}
{'loss': 2.1718, 'learning_rate': 2.00296

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.2002689838409424, 'eval_accuracy': 0.458, 'eval_runtime': 4.1429, 'eval_samples_per_second': 241.375, 'eval_steps_per_second': 30.172, 'epoch': 2.0}
{'loss': 2.1822, 'learning_rate': 1.706231454005935e-05, 'epoch': 2.08}
{'loss': 2.173, 'learning_rate': 1.5578635014836794e-05, 'epoch': 2.16}
{'loss': 2.1519, 'learning_rate': 1.4094955489614246e-05, 'epoch': 2.24}
{'loss': 2.1788, 'learning_rate': 1.2611275964391692e-05, 'epoch': 2.32}
{'loss': 2.164, 'learning_rate': 1.112759643916914e-05, 'epoch': 2.4}
{'loss': 2.1724, 'learning_rate': 9.643916913946588e-06, 'epoch': 2.48}
{'loss': 2.1537, 'learning_rate': 8.160237388724036e-06, 'epoch': 2.56}
{'loss': 2.1502, 'learning_rate': 6.676557863501484e-06, 'epoch': 2.64}
{'loss': 2.1428, 'learning_rate': 5.192878338278932e-06, 'epoch': 2.72}
{'loss': 2.1413, 'learning_rate': 3.7091988130563796e-06, 'epoch': 2.8}
{'loss': 2.1561, 'learning_rate': 2.225519287833828e-06, 'epoch': 2.88}
{'loss': 2.1694, 'learning_rate': 7.4183976

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.15342378616333, 'eval_accuracy': 0.411, 'eval_runtime': 3.9376, 'eval_samples_per_second': 253.959, 'eval_steps_per_second': 31.745, 'epoch': 3.0}
{'train_runtime': 157.3047, 'train_samples_per_second': 76.285, 'train_steps_per_second': 2.384, 'train_loss': 2.2188035736083984, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=2.2188035736083984, metrics={'train_runtime': 157.3047, 'train_samples_per_second': 76.285, 'train_steps_per_second': 2.384, 'train_loss': 2.2188035736083984, 'epoch': 3.0})

OutOfMemoryError: CUDA out of memory.   
First we tried to use "google/efficientnet-b7" model, but it too large to run on a single GPU. We then replace it with model "google/efficientnet-b0".

Model is learning as the loss is decreasing, however very slowly. why is that, and how we can make the learning faster?

In [ ]:
training_args = TrainingArguments(
    output_dir="cifar10_efficientnet",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8, # memory error with 16
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=cifar10dataset["train"],
    eval_dataset=cifar10dataset["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()